In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [1]:
!pip install selenium
!pip install webdriver_manager
# webdriver_manager는 4버전 이상에서 크롬 버전을 자동으로 관리

  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.4 MB 1.1 MB/s eta 0:00:09
    --------------------------------------- 0.2/9.4 MB 2.0 MB/s eta 0:00:05
   -- ------------------------------------- 0.5/9.4 MB 3.6 MB/s eta 0:00:03
   -- ------------------------------------- 0.5/9.4 MB 3.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/9.4 MB 5.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/9.4 MB 5.5 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.4 MB 5.2 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.4 MB 5.0 MB/s eta 0:00:02
   --------- ------------------------------ 2.1/9.4 MB 5.0 MB/s eta 0:00:02
   ---------- ----------------------------- 2.4/9.4 MB 5.2 MB/s eta 0:00:02
   ---------- --------------------------

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

NameError: name 'selenium' is not defined

# Crawling

## 동적크롤링

reference: https://selenium-python.readthedocs.io/index.html

### 설치 방법

```cmd
pip install selenium
```

Chrome Driver: https://chromedriver.chromium.org/downloads

### 사용법

기본적인 사용 방법은 아래와 같음
```python
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
 
driver = webdriver.Chrome(executable_path='chromedriver') # executable_path: chromedriver path
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))    # 4버전 이상
# driver = webdriver.Firefox()

driver.get(url="http://www.naver.com")

driver.close()
```

In [5]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 
# 이 화면은 끄지말고 켜놓을 것

In [6]:
driver.get(url="https://finance.daum.net/domestic/market_cap")

In [ ]:
bs = BeautifulSoup(driver.page_source, 'lxml')

In [12]:
bs.select('div.box_contents')

[<div class="box_contents"><div style="display: block;"><table>
 <colgroup>
 <col width="44"/>
 <col width="*"/>
 <col width="84"/>
 <col width="84"/>
 <col width="84"/>
 <col width="84"/>
 <col width="94"/>
 <col width="84"/>
 </colgroup>
 <thead>
 <tr>
 <th class="lAlign">순위</th>
 <th class="lAlign">종목명</th>
 <th class="rAlign">현재가</th>
 <th class="rAlign">전일비</th>
 <th class="rAlign">
 <a class="" data-field-name="changeRate" href="javascript:void(0)">등락률 <i>-</i></a>
 </th>
 <th class="rAlign">
 <a class="" data-field-name="marketCap" href="javascript:void(0)">시가총액 <i>-</i></a>
 </th>
 <th class="rAlign">
 <a class="" data-field-name="listedShareCount" href="javascript:void(0)">상장주식수 <i>-</i></a>
 </th>
 <th class="rAlign last">
 <a class="" data-field-name="foreignOwnRate" href="javascript:void(0)">외국인 <i>-</i></a>
 </th>
 </tr>
 </thead>
 <tbody><tr class="first"><td class="first"><span class="time">1</span></td>
 <td><a class="txt" href="/quotes/A005930">삼성전자</a></td>
 <td>
 <sp

#### Locating Elements

find_element: 조건에 해당하는 하나의 값 반환  
find_elements: 조건에 해당하는 다수의 값 반환

element를 찾는 method는 아래와 같음
- by_ID
- by_xpath
- by_link_text
- by_partial_link_text
- by_name
- by_tag_name
- by_class_name
- by_css_selector

```python
element = driver.find_element(By.XPATH, x_path)
element = driver.find_element_by_xpath(x_path)

element = driver.find_element(By.CSS_SELECTOR, css)
element = driver.find_element_by_css_selector(css)
```

해당 값을 못 찾은 경우 NoSuchElementException 발생

In [ ]:
driver.find_element(By.XPATH, '/html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[1]').click()


In [100]:
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[1] # 1페이지에서 2페이지 갈때 xpath
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[2] # 2페이지에서 3페이지 갈때 xpath

driver.get(url="https://finance.daum.net/domestic/market_cap")
bs = BeautifulSoup(driver.page_source, 'lxml')

In [71]:
colunm = [tag.text.strip().replace(' -','') for tag in bs.select('div.box_contents')[0].select('tr')[0].select('th')]
colunm
stock_info =[]
total_stock_info = []
for row in range(1, 31):
    stock_info.append([tag.text.strip().replace('-','') for tag in bs.select('div.box_contents')[0].select('tr')[row].select('th')])

for page in tqdm(range(1, 10)):
    stock_info =[]
    driver.find_element(By.XPATH, f'/html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[{page}]').click()
    time.sleep(1)
    bs = BeautifulSoup(driver.page_source, 'lxml')

    for row in range(1, 31):
        stock_info.append([tag.text.strip().replace('-','') for tag in bs.select('div.box_contents')[0].select('tr')[row].select('th')])

    total_stock_info.extend(stock_info)

 89%|████████▉ | 8/9 [00:10<00:01,  1.32s/it]


KeyboardInterrupt: 

In [48]:
len(total_stock_info)

270

In [8]:
bs = BeautifulSoup(driver.page_source, 'lxml')
colunm = [tag.text.strip().replace(' -','') for tag in bs.select('div.box_contents')[0].select('tr')[0].select('th')]
colunm
stock_info =[]
total_stock_info = []


stock_info =[]
total_stock_info = []
# 71페이지
for row in range(1, 31):
    stock_info.append([tag.text.strip() for tag in bs.select('div.box_contents')[0].select('tr')[row].select('th')])
total_stock_info.extend(stock_info)

# 72페이지부터
for page in tqdm(range(7, 12)):
    
    driver.find_element(By.XPATH, f'/html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[{page}]').click()
    
    time.sleep(2)
    bs = BeautifulSoup(driver.page_source, 'lxml')

    try:
        for row in range(1, 31):
            stock_info.append([tag.text.strip() for tag in bs.select('div.box_contents')[0].select('tr')[row].select('th')])
    except:
        pass

    total_stock_info.extend(stock_info)

100%|██████████| 5/5 [00:10<00:00,  2.15s/it]


In [9]:
total_stock_info

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [73]:
/html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[7]

['순위', '종목명', '현재가', '전일비', '등락률', '시가총액', '상장주식수', '외국인']

##### Send Keys
```python
driver.find_element_by_xpath(xpath).send_keys(id)
driver.find_element_by_xpath(xpath).send_keys(password)
driver.find_element_by_xpath(xpath).click()
```

외에도 Keys 내의 다양한 커맨드 입력 가능
```
from selenium.webdriver.common.keys import Keys
```
|Command          |Action |
|-----------------|-------|
|Keys.ENTER       |엔터    |
|Keys.RRETURN     |       |
|Keys.SPACE       |스페이스 |
|Keys.ARROW_UP    |화살표   |
|Keys.ARROW_DOWN  |       |
|Keys.ARROW_LEFT  |       |
|Keys.ARROW_RIGHT |       |
|Keys.BACK_SPACE  |지우기   |
|Keys.DELETE      |       |
|Keys.CONTROL     |Ctrl   |
|Keys.ALT         |Alt    |
|Keys.SHIFT       |Shift  |
|Keys.TAB         |Tab    |
|Keys.PAGE_UP     |Page-Up|
|Keys.PAGE_DOWN   |Page-Down|
|Keys.TAB         |Tab    |
|Keys.F1 ~ Keys.F9|F1~F9  |
|Keys.ESCAPE      |ESC    |
|Keys.HOME        |Home   |
|Keys.INSERT      |Insert |
|...              |...    |

In [13]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/div/input').send_keys('월급')
driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/div/input').send_keys(Keys.ENTER)
# 자동으로 입력을 해줌

In [15]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/div/input').send_keys('월급')
driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/button').click()



In [109]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 

In [21]:
driver.get(url = 'https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
# BeautifulSoup(driver.page_source, 'lxml') 윗줄이랑 같이 사용하면 작동을 안하는데 이유는 윗줄이 끝나기전에 아랫줄이 실행되서 이다
# .sleep()을 사용해서 해결

In [23]:
bs = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
bs

In [ ]:
driver.get(url = 'https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]')

In [85]:
driver.get(url = 'https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]'))
)
bs = BeautifulSoup(driver.page_source, 'lxml')


In [86]:
colunm = [title.text for title in bs.select('thead th')]
colunm

['공개년월',
 '기업집단명',
 '소속회사명',
 '대표자',
 '설립일',
 '계열편입일',
 '업종코드',
 '영위업종',
 '종업원수',
 '결산기일',
 '결산주총일',
 '기업공개일',
 '자산총액',
 '부채총액',
 '자본총액',
 '자본금',
 '매출액',
 '당기순이익',
 '구분',
 '법인등록번호',
 '사업자등록번호']

In [87]:
tbody = bs.select('tbody tr')
tbody[1].text.strip().split('\n')

['202305',
 '지에스',
 '(유)가승개발',
 '최광옥',
 '2016/01/21',
 '2016/04/01',
 'R9112',
 '골프장 및 스키장 운영업',
 '0',
 '2022/12/31',
 '2023/03/27',
 '',
 '48,952',
 '47,666',
 '1,286',
 '100',
 '0',
 '-3,699',
 '일반회사',
 '1345140003989',
 '1198800340']

In [92]:

driver.get(url = 'https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]'))
)
bs = BeautifulSoup(driver.page_source, 'lxml')

company_info = []
tbody = bs.select('tbody tr')

for i in range(10):
    company_info.append(tbody[i].text.strip().split('\n')[:9])
    print(company_info[i])

for page in tqdm(range(3, 12)):
    
    driver.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[{page}]')
    bs = BeautifulSoup(driver.page_source, 'lxml')
                
    try:
        for i in range(10):
            company_info.append(tbody[i].text.strip().split('\n')[:9])
    except Exception as error:
        print(error)
        





['202405', '지에스', '(유)가승개발', '최광옥', '2016/01/21', '2016/01/21', 'R9112', '골프장 및 스키장 운영업', '0']
['202305', '지에스', '(유)가승개발', '최광옥', '2016/01/21', '2016/04/01', 'R9112', '골프장 및 스키장 운영업', '0']
['202205', '지에스', '(유)가승개발', '최광옥', '2016/01/21', '2016/04/01', 'R91', '스포츠 및 오락관련 서비스업', '4']
['202405', '카카오', '(유)가승개발', '최광옥', '2016/01/21', '2021/02/01', 'R9112', '골프장 및 스키장 운영업', '0']
['202305', '카카오', '(유)가승개발', '최광옥', '2016/01/21', '2021/02/01', 'R9112', '골프장 및 스키장 운영업', '0']
['202205', '카카오', '(유)가승개발', '최광옥', '2016/01/21', '2021/02/01', 'R91', '스포츠 및 오락관련 서비스업', '4']
['202305', '중앙', '(유)그린에너지매니지먼트아시아', '최준용', '2021/05/24', '2021/06/01', 'L6811', '부동산 임대업', '0']
['202205', '에스케이', '(유)나눔푸드', '김치훈', '2008/05/27', '2022/05/01', 'C10', '식료품 제조업', '16']
['202405', '네이버', '(유)나라미', '최선규', '2022/01/14', '2024/04/01', 'K6499', '그 외 기타 금융업', '1']
['202405', '에스케이', '(유)로지소프트', '이영수', '2002/08/09', '2022/10/01', 'J5822', '시스템ㆍ응용 소프트웨어 개발 및 공급업', '28']


100%|██████████| 9/9 [00:00<00:00, 21.51it/s]


In [96]:
# 1페이지

driver.get(url = 'https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]'))
)
bs = BeautifulSoup(driver.page_source, 'lxml')
bs.select('table')

company_info = []

for i, row in enumerate(bs.select('table')[0].select('tr')):
    if i  == 0 :
        element = [tag.text for tag in row.select('th')[:8]]
    else:
        element = [tag.text for tag in row.select('td')[:8]]

    company_info.append(element)

# 2페이지 부터

driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]')

for page in tqdm(range(4, 12)):
    WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]'))
            )
    bs = BeautifulSoup(driver.page_source, 'lxml')
    bs.select('table')
    
    for i, row in enumerate(bs.select('table')[0].select('tr')):
        if i  == 0 :
            element = [tag.text for tag in row.select('th')[:8]]
        else:
            element = [tag.text for tag in row.select('td')[:8]]

        company_info.append(element)
    
    if page != 12:
    
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[{page}]'))
        ).find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[{page}]').click()
                    
    

 12%|█▎        | 1/8 [00:01<00:09,  1.31s/it]


ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:fn_pagination('...');">4</a> is not clickable at point (363, 640). Other element would receive the click: <div class="bgshadow"></div>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00B48923+23283]
	(No symbol) [0x00B0E934]
	(No symbol) [0x00A40733]
	(No symbol) [0x00A891C7]
	(No symbol) [0x00A875D9]
	(No symbol) [0x00A854CB]
	(No symbol) [0x00A84AAF]
	(No symbol) [0x00A799EF]
	(No symbol) [0x00AA441C]
	(No symbol) [0x00A79485]
	(No symbol) [0x00AA46B4]
	(No symbol) [0x00ABCB8D]
	(No symbol) [0x00AA41B6]
	(No symbol) [0x00A78017]
	(No symbol) [0x00A7890D]
	GetHandleVerifier [0x00C3A5F3+1013699]
	GetHandleVerifier [0x00C43E4C+1052700]
	GetHandleVerifier [0x00C3D4B4+1025668]
	GetHandleVerifier [0x00B6EA2B+179195]
	(No symbol) [0x00B16833]
	(No symbol) [0x00B13198]
	(No symbol) [0x00B13337]
	(No symbol) [0x00B0B4BE]
	BaseThreadInitThunk [0x76717BA9+25]
	RtlInitializ

In [91]:
company_info[-1]

['202405',
 '에스케이',
 '(유)로지소프트',
 '이영수',
 '2002/08/09',
 '2022/10/01',
 'J5822',
 '시스템ㆍ응용 소프트웨어 개발 및 공급업',
 '28']

In [ ]:
/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[11]
/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]
element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3]'))
)

##### Drag and Drop

```python
from selenium.webdriver import ActionChains

element = driver.find_element(By.NAME, "source")
target = driver.find_element(By.NAME, "target")

action_chains = ActionChains(driver)
action_chains.drag_and_drop(element, target).perform()
```

##### Moving between Windows

```python
previous_window = driver.window_handles[0]
new_window = driver.window_handles[1]
driver.switch_to.window(new_window)
```


##### Moving

```python
driver.forward()
driver.back()
```

#### Waiting

Selenium은 실행 중인 driver를 이용하여 정보를 추출  
driver에 데이터 로딩이 완료되지 않을 경우 데이터 수집이 불가  
따라서 페이지 이동 등의 작업이 들어가면 다음 작업이 완료될 때까지 대기해야만 함

##### Time

가장 기본적인 방법으로 대기 시간을 time 모듈을 통해 명시

```python
import time
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url="http://www.naver.com")
time.time(5)
driver.close()
```

##### Implicit Wait

위의 time에서는 2가지 문제가 존재
1. 모든 반응형 동작마다 time을 걸어 웹 로딩 대기
2. time을 5초로 설정했지만 실제 그보다 동작이 빨리 끝날 경우 불필요한 시간 대기

위의 문제를 해결하고자 implicit wait을 이용  
이는 driver의 옵션 설정으로 반응형 동작에서 최대 허용 대기 시간을 설정  
만약 허용 시간 이내에 로드가 완료될 경우 다음 작업을 바로 진행

```python
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(time_to_wait=5)
driver.get(url="http://www.naver.com")
driver.close()

```

등장할때 까지 최대한 기다리다가 넘어가면 에러냄


##### Explicit Wait

implicit wait에서는 driver가 로딩을 대기하는 시간을 설정하여 5초를 설정하였어도 이전에 로드가 완료되면 다음 작업의 진행이 가능하였음  
하지만 위 경우도 문제가 있는데 5초가 지나도 내가 원하는 데이터가 로드가 안 되었을 경우 데이터를 수집할 수 없음  
따라서 특정 값이 로드될 때까지 기다렸다가 그 값이 로드되면 다음 작업을 진행할 필요성이 발생  
이 때 이용하는 게 explicit wait

```python
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path='chromedriver') 
driver.get(url="http://www.naver.com")

try:
    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME , 'paging'))
    )
finally:
    driver.quit()

driver.close()

```

until: 조건이 False인 동안에 계속 실행  
not_until: 조건이 True인 동안에 계속 실행

#### ActionChains

연속 동작을 수행하기 위함

ex) control + c
```python
ActionChains(driver).key_down(Keys.CONTROL).send_keys('c').key_up(Keys.CONTROL).perform()
```

##### Drag and Drop

```python
from selenium.webdriver import ActionChains

element = driver.find_element(By.NAME, "source")
target = driver.find_element(By.NAME, "target")

action_chains = ActionChains(driver)
action_chains.drag_and_drop(element, target).perform()
```

#### Others

##### Options
```python
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920,1080')

driver = webdriver.Chrome(executable_path, options=options)
```

##### Alert

경고창 발생 시 이에 수락, 거절 등의 행동을 취할 수 있음

```python
from selenium.webdriver.common.alert import Alert

Alert(driver).accept() # 수락
Alert(driver).dismiss() # 거절
Alert(driver).send_keys(keysToSend=key) # 특정 키를 보낼 수 있음
```

##### Scroll Down
페이지의 최하단으로 이동
```python
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
```
<br>

특정 태그가 등장할 때까지 이동
```python
from selenium.webdriver import ActionChains

some_tag = driver.find_element_by_id('gorio')
ActionChains(driver).move_to_element(some_tag).perform()
```

##### Minimize/Maximize

```python
driver.minimize_window()
driver.maximize_window()
```

##### Screen Shot
```python
driver.save_screenshot('screenshot.png')
```

# Practice

In [148]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 

In [149]:
driver.get('https://dart.fss.or.kr/main.do')

driver.find_element(
    By.XPATH,
    '/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[2]/div/span[2]/input'
    ).send_keys('삼성전자')

driver.find_element(
    By.XPATH,
    '/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[2]/div/span[2]/input'
    ).send_keys(Keys.ENTER)

# 반기보고서 클릭
WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a'))
).find_element(
    By.XPATH,
    '/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a'
).click()



In [150]:
driver.window_handles
driver.switch_to.window(driver.window_handles[-1])

In [151]:
bs = BeautifulSoup(requests.get('https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20240814003284').text, 'lxml')

In [160]:
bs = BeautifulSoup(requests.get('https://dart.fss.or.kr/report/viewer.do?rcpNo=20240814003284&dcmNo=10070946&eleId=20&offset=284259&length=22788&dtd=dart4.xsd').text, 'lxml')

In [163]:
bs.select('a')[0].text

'【 대표이사 등의 확인 】'

In [119]:
driver.switch_to.window(driver.window_handles[-1])

In [120]:
driver.find_element(
    By.XPATH,
    '/html/body/div[3]/div/div[2]/div[1]/div[2]/ul/ul/li[5]/ul/li[2]/a' 
    ).click()

In [121]:
(
    BeautifulSoup(driver.page_source, 'lxml')
    .select('table')
)

[]

In [ ]:
https://dart.fss.or.kr/report/viewer.do?rcpNo=20240814003284&dcmNo=10070946&eleId=2&offset=27926&length=425&dtd=dart4.xsd
https://dart.fss.or.kr/report/viewer.do?rcpNo=20240814003284&dcmNo=10070946&eleId=20&offset=284259&length=22788&dtd=dart4.xsd

In [122]:
driver.close()

In [123]:
driver.switch_to.window(driver.window_handles[-1])

## Dart

In [127]:
url = 'https://finance.naver.com/sise/sise_market_sum.naver'

response = requests.get(url)
bs = BeautifulSoup(response.text, 'lxml')



<a class="tltle" href="/item/main.naver?code=005930">삼성전자</a>

In [132]:
name = [name.text for name in bs.select('a.tltle')]
names = name[:10]
names

['삼성전자',
 'SK하이닉스',
 'LG에너지솔루션',
 '삼성바이오로직스',
 '현대차',
 '삼성전자우',
 '셀트리온',
 '기아',
 'KB금융',
 '신한지주']

In [134]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) 

In [135]:
driver.get('https://dart.fss.or.kr/main.do')

total_index = []

for name in names:
    driver.find_element(
        By.XPATH,
        '/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[2]/div/span[2]/input'
        ).send_keys(name)

    driver.find_element(
        By.XPATH,
        '/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[2]/div/span[2]/input'
        ).send_keys(Keys.ENTER)

    # 반기보고서 클릭
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a'))
    ).find_element(
        By.XPATH,
        '/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a'
    ).click()


    driver.window_handles
    driver.switch_to.window(driver.window_handles[-1])


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[2]/div/span[2]/input"}
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00B48923+23283]
	(No symbol) [0x00B0E934]
	(No symbol) [0x00A40733]
	(No symbol) [0x00A8326F]
	(No symbol) [0x00A834AB]
	(No symbol) [0x00ABEE42]
	(No symbol) [0x00AA4464]
	(No symbol) [0x00ABCB8D]
	(No symbol) [0x00AA41B6]
	(No symbol) [0x00A78017]
	(No symbol) [0x00A7890D]
	GetHandleVerifier [0x00C3A5F3+1013699]
	GetHandleVerifier [0x00C43E4C+1052700]
	GetHandleVerifier [0x00C3D4B4+1025668]
	GetHandleVerifier [0x00B6EA2B+179195]
	(No symbol) [0x00B16833]
	(No symbol) [0x00B13198]
	(No symbol) [0x00B13337]
	(No symbol) [0x00B0B4BE]
	BaseThreadInitThunk [0x76717BA9+25]
	RtlInitializeEx